In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
%matplotlib inline

from pathlib import Path

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

In [ ]:
#LOADING DATA 
train_values = pd.read_csv('train_values.csv', index_col='building_id')
train_labels = pd.read_csv('train_labels.csv', index_col='building_id')

train_values.dtypes

FileNotFoundError: ignored

In [ ]:
#EXPLORING DATA 
(train_labels.damage_grade
             .value_counts()
             .sort_index()
             .plot.bar(title="Number of Buildings with Each Damage Grade"))

selected_features = ['foundation_type', 
                     'area_percentage', 
                     'height_percentage',
                     'count_floors_pre_eq',
                     'land_surface_condition',
                     'has_superstructure_cement_mortar_stone']

train_values_subset6 = train_values[selected_features]

sns.pairplot(train_values_subset.join(train_labels), 
             hue='damage_grade')

In [ ]:
#BUILDING THE MODEL 
# Random Forest
from sklearn.preprocessing import StandardScaler

# the model
from sklearn.ensemble import RandomForestClassifier

# for combining the preprocess with model training
from sklearn.pipeline import make_pipeline

# for optimizing the hyperparameters of the pipeline
from sklearn.model_selection import GridSearchCV

In [ ]:
# names the steps in your pipeline as a lowercase version of whatever the object name is
#we can just take out the standardscaler
#look at the documentation of the standardscaler - numeric, categorical .... 
#if subset model is similar, slightly lower we knows its working well 

train_values_full = pd.get_dummies(train_values)
#Testing All Features
pipe = make_pipeline(RandomForestClassifier(random_state=2018))
pipe

param_grid = {'randomforestclassifier__n_estimators': [50, 100],
              'randomforestclassifier__min_samples_leaf': [1, 5]}
# test a few different models using GridSearchCV              
gs = GridSearchCV(pipe, param_grid, cv=5)

gs.fit(train_values_full, train_labels.values.ravel())

gs.best_params_

from sklearn.metrics import f1_score
# sample f1 score
in_sample_preds = gs.predict(train_values_full)

# Compute the in-sample F1 score
in_sample_f1 = f1_score(train_labels, in_sample_preds, average='micro')
print("In-sample F1 score:", in_sample_f1)

# Create a cross-tab of the in-sample predictions and the true labels
cross_tab = pd.crosstab(in_sample_preds, train_labels.values.ravel())
print(cross_tab)

In [ ]:
#testing 6 features (given) 
import pandas as pd
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score

train_values_subset6 = train_values[selected_features]
train_values_subset6 = pd.get_dummies(train_values_subset6)
# Create the pipeline
pipe = make_pipeline(RandomForestClassifier(random_state=2018))

# Define the hyperparameter grid
param_grid = {'randomforestclassifier__n_estimators': [50, 100],
              'randomforestclassifier__min_samples_leaf': [1, 5]}

# Create the GridSearchCV object
gs = GridSearchCV(pipe, param_grid, cv=5)

# Fit the model to the training data
gs.fit(train_values_subset6, train_labels.values.ravel())

# Print the best parameters
print("Best parameters:", gs.best_params_)

# Get the in-sample predictions
in_sample_preds = gs.predict(train_values_subset6)

# Compute the in-sample F1 score
in_sample_f1 = f1_score(train_labels, in_sample_preds, average='micro')
print("In-sample F1 score:", in_sample_f1)

# Create a cross-tab of the in-sample predictions and the true labels
cross_tab = pd.crosstab(in_sample_preds, train_labels.values.ravel())
print(cross_tab)


In [ ]:
#PREPROCESS THE DATA
# train_values_full = pd.get_dummies(train_values) 
# train_values_subset22 = pd.get_dummies(train_values_subset22)
# train_values_subset10 = pd.get_dummies(train_values_subset10)
# train_values_subset6 = pd.get_dummies(train_values_subset6)


In [ ]:
# RIDGE  
#should pull out a list of variables and coefficient sizes - for it to work everything has to be standardized 
#coefficients have meaning after scaling 0-1 or 1-100 
#reduces coefficient sizes for unimportant variables and increases coefficient sizes for important variables - lasso will push to zero ridge will not
#advantage allows it to maintain clusters of variables that are correlated to each other - useful for dataset with highly correlated variables
# if we reduce and score is still high - model is solid! 
# use ridge for variable selection - pull out coefficient list - an argument inside the model after fitting
#cross tabs
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split

# Load the data
X = train_values_full
y = train_labels['damage_grade']

# Split the data into a training set and a test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the scaler
scaler = StandardScaler()

# Standardize the training and test data
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Initialize the ridge model
model = Ridge(alpha=.5)

# Fit the model to the standardized training data
model.fit(X_train_scaled, y_train)

# Get the list of coefficients
coefficients = model.coef_

# Print the coefficients
print(coefficients)

# Evaluate the model on the standardized test data
y_pred = model.predict(X_test_scaled)
test_accuracy = model.score(X_test_scaled, y_test)
print("Test accuracy:", test_accuracy)


In [ ]:
def get_top_coefs(model, X, num_to_keep=0.10):
    # Get the list of coefficients and the list of feature names
    coefficients = model.coef_
    feature_names = list(X.columns)

    # Zip the coefficients and feature names together
    coef_feature_tuples = list(zip(coefficients, feature_names))

    # Sort the tuples by absolute value of the coefficient
    sorted_tuples = sorted(coef_feature_tuples, key=lambda x: abs(x[0]))
    
    # Keep the top coefficients by absolute value
    num_to_keep = int(len(sorted_tuples) * num_to_keep)
    top_coefs = sorted_tuples[-num_to_keep:]
    
    return top_coefs

# Get the top 15% of coefficients
top_coefs = get_top_coefs(model, X, num_to_keep=0.10)

# Print the top coefficients
print(top_coefs)
top10_coefs = ['has_secondary_use', 
                     'foundation_type_r', 
                     'foundation_type_w',
                     'has_superstructure_cement_mortar_brick',
                     'has_secondary_use_use_police',
                     'plan_configuration_q',
                     'has_superstructure_mud_mortar_stone', 
                     'has_secondary_use_school', 
                     'has_superstructure_stone_flag',
                     'has_superstructure_rc_engineered' 
               ]

In [ ]:
#testing top 10 Coefs from Ridge Regression  
train_values_subset10 = train_values_full[top10_coefs]
train_values_subset10 = pd.get_dummies(train_values_subset10)

pipe = make_pipeline(RandomForestClassifier(random_state=2018))
pipe

param_grid = {'randomforestclassifier__n_estimators': [50, 100],
              'randomforestclassifier__min_samples_leaf': [1, 5]}
# test a few different models using GridSearchCV              
gs = GridSearchCV(pipe, param_grid, cv=5)

gs.fit(train_values_subset10, train_labels.values.ravel())

gs.best_params_

from sklearn.metrics import f1_score
# sample f1 score
in_sample_preds = gs.predict(train_values_subset10)

# Compute the in-sample F1 score
in_sample_f1 = f1_score(train_labels, in_sample_preds, average='micro')
print("In-sample F1 score:", in_sample_f1)

# Create a cross-tab of the in-sample predictions and the true labels
cross_tab = pd.crosstab(in_sample_preds, train_labels.values.ravel())
print(cross_tab)

In [ ]:
# BACKWARDS SELECTION 
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

# Load the data
X = train_values_full
y = train_labels['damage_grade']

# Split the data into a training set and a test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the model
model = LinearRegression()

# Initialize the list of features to include in the model
features = list(X_train.columns)

# Initialize the list of scores for each iteration
scores = []

# Iterate over a range of number of features to remove
for num_to_remove in range(len(features)):
    # Fit the model with the current set of features
    model.fit(X_train[features], y_train)
    
    # Get the score for the current set of features
    score = model.score(X_test[features], y_test)
    
    # Append the score to the list of scores
    scores.append(score)

    # If this is the best score so far, store the features
    if score == max(scores):
        best_features = features.copy()

    # Print the best set of features and the corresponding score
print("Best set of features:", best_features)
print("Best score:", max(scores))


In [ ]:
#testing backwards selection (22 Coef)
train_values_subset22 = train_values_full[best_features]
train_values_subset22 = pd.get_dummies(train_values_subset22)

pipe = make_pipeline(RandomForestClassifier(random_state=2018))
pipe

param_grid = {'randomforestclassifier__n_estimators': [50, 100],
              'randomforestclassifier__min_samples_leaf': [1, 5]}
# test a few different models using GridSearchCV              
gs = GridSearchCV(pipe, param_grid, cv=5)

gs.fit(train_values_subset22, train_labels.values.ravel())

gs.best_params_

from sklearn.metrics import f1_score
# sample f1 score
in_sample_preds = gs.predict(train_values_subset22)

# Compute the in-sample F1 score
in_sample_f1 = f1_score(train_labels, in_sample_preds, average='micro')
print("In-sample F1 score:", in_sample_f1)

# Create a cross-tab of the in-sample predictions and the true labels
cross_tab = pd.crosstab(in_sample_preds, train_labels.values.ravel())
print(cross_tab)